In [1]:
import re
import os
from tqdm import trange, tqdm

In [2]:
fname_ml="/home/projects/semmetrix/chilecto/corp/SketchEngine/zhTenTen_corpus/example_zhTenTen.vert"

In [3]:
fname_tw="/home/projects/semmetrix/chilecto/corp/SketchEngine/taiwan_corpus/example_chinese_taiwan.vert"

# Def

In [4]:
def get_url_list(fname):
    with open(fname, 'r') as f:
        f_str = f.read()
        
    all_match = re.finditer(r'<doc (.+?)>', f_str)
    n_doc = 0
    n_url = 0
    url_list = []
    for match in all_match:
        n_doc += 1
        url_match = re.findall(r'"http://(\S+?)"', match.group())
        url_match = list(set(url_match))
        if len(url_match) == 0:
            print(f'URL not found for {match.group()}')
        else:
            if len(url_match) > 1:
                print(f'More than one URL found for {match.group()}')

            url_list.extend(url_match)

    print(f'{len(url_list)} URL found in {n_doc} docs')

    return url_list

In [5]:
# get_url_list(fname)

In [6]:
t = re.search(r'<doc (.+?)>(.+?)</doc>', f_str, re.DOTALL)
len(t.group(2).split('\n'))

NameError: name 'f_str' is not defined

In [7]:
def get_file_data(fname, prt=True):

    file_data = []
    with open(fname, 'r') as f:
        f_str = f.read()

    doc_match = re.finditer(r'<doc (.+?)>(.+?)</doc>', f_str, re.DOTALL)
    n_doc = 0
    n_url = 0
    for doc in doc_match:
        n_doc += 1
        info_str = doc.group(1)
        url_match = re.findall(r'"https?://(\S+?)"', info_str)
        url_match = list(set(url_match))
        if len(url_match) == 0:
            print(f'URL not found for {info_str}')
        else:
            if len(url_match) > 1 and prt:
                print(f'More than one URL found for {info_str}')
                
        n_url += len(url_match)       
        # number of words
        word_list = doc.group(2).split('\n')
        n_word = 0
        for word in word_list:
            if len(word)==0 or word[0] == '<':
                continue
            else:
                n_word += 1        

        doc_data = {'idx': n_doc-1, 'url_list': url_match, 'n_word': n_word}
        
        file_data.append(doc_data)
    if prt:
        print(f'{n_url} URL found in {n_doc} docs')

    return file_data

In [21]:
def search_url(file_data, search_list):
    total_word = 0
    search_word = 0
    for data in file_data:
        count = False
        for e in search_list:
            for url in data['url_list']:
                if e in url:
                    count = True
        if count:
            search_word += data['n_word']
        total_word += data['n_word']
        
    p = float(search_word)/float(total_word)*100.0
    print(f'{search_word:,}/{total_word:,} ({p:4.2f}%)')

In [9]:
def process_fnames(fnames):
    with open(fnames, 'r') as f:
        file_list = f.readlines()
    
    output = []
    for file in file_list:
        output.extend(get_file_data(file.replace('\n', '')))
        
    return output

In [10]:
def process_file_list(file_list):
    """
    """
    output = []
    for file in tqdm(file_list):
        output.extend(get_file_data(file.replace('\n', ''), prt=False))
        
    return output

In [11]:
def gen_file_list(root_dir):
    """
    """
    file_list = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for file in filenames:
            if file[-4:] == 'vert':
                file_list.append(os.path.join(dirpath, file))
            
    return file_list

In [12]:
file_data_ml = get_file_data(fname_ml)

More than one URL found for contenttype="text/html" guessed_charset="gb2312" ip="61.132.133.25" lang_filter_score="0.77" length="40095" timestamp="20110427135532" url="http://www1.ahedu.gov.cn/wwmx/" http_accept_ranges="bytes" http_connection="close" http_content_length="39765" http_content_location="http://www1.ahedu.gov.cn/wwmx/index.html" http_content_type="text/html" http_date="Wed, 27 Apr 2011 13:55:36 GMT" http_etag="&quot;98ef7a31a7e3cb1:767&quot;" http_last_modified="Wed, 16 Mar 2011 06:55:53 GMT" http_server="Microsoft-IIS/6.0" http_x_powered_by="ASP.NET"
778 URL found in 777 docs


In [35]:
search_url(file_data_ml, ['forum','bbs','blog'])

25,896/904,543 (2.86%)


In [ ]:
file_data_tw = get_file_data(fname_tw)

In [ ]:
search_url(file_data_tw, ['forum','bbs','blog'])

In [ ]:
for item in file_data_ml:
    print(item['url_list'])

In [ ]:
data_fnames_ml = process_fnames('/home/projects/semmetrix/chilecto/corp/SketchEngine/ml_rand_select_file.fnames')

In [ ]:
data_fnames_tw = process_fnames('/home/projects/semmetrix/chilecto/corp/SketchEngine/tw_rand_select_file.fnames')

In [ ]:
search_url(data_fnames_ml, ['forum','bbs','blog'])

In [ ]:
search_url(data_fnames_ml, ['.gov.','news','.edu.'])

In [ ]:
search_url(data_fnames_tw, ['forum','bbs','blog'])

In [ ]:
search_url(data_fnames_tw, ['.gov.','news','.edu.'])

In [13]:
tw_dir = '/home/projects/semmetrix/chilecto/corp/SketchEngine/taiwan_corpus/processed/'
ml_dir = '/home/projects/semmetrix/chilecto/corp/SketchEngine/zhTenTen_corpus/processed/'

In [14]:
tw_data = process_file_list(gen_file_list(tw_dir))

100%|██████████| 1186/1186 [02:27<00:00,  8.63it/s]


In [15]:
ml_data = process_file_list(gen_file_list(ml_dir))

100%|██████████| 23364/23364 [52:10<00:00,  7.40it/s]  


In [29]:
search_url(tw_data, ['.blog.'])

1,055,142/349,130,587 (0.30%)


In [36]:
search_url(tw_data, ['blog'])

28,042,814/349,130,587 (8.03%)


In [26]:
search_url(tw_data, ['blog','bbs','forum'])

67,571,325/349,130,587 (19.35%)


In [30]:
search_url(ml_data, ['.blog.'])

34,110,847/6,093,932,135 (0.56%)


In [34]:
search_url(ml_data, ['blog'])

76,677,540/6,093,932,135 (1.26%)


In [24]:
search_url(ml_data, ['blog','bbs','forum'])

283,625,620/6,093,932,135 (4.65%)
